In [2]:
#import gym
import math
import random
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as T

from itertools import count
from collections import namedtuple

In [3]:
# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if gpu is to be used
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from scipy.spatial.distance import euclidean
import numpy as np
from numpy.random import randint
from itertools import permutations
from random import shuffle

"""
Actions:
    -Draw vertex
    -Measure distance
    -Measure angle
"""
# DEFINE ENVIRONMENT
class GeometryEnvironment():
    def __init__(self):
        self.reset()

    def place_vertex(self, bead_index, vertex_index):
        self.bead_set[bead_index] = vertex_index

    def measure_distance(self, i, j):
        u = self.vertex_set[i]
        v = self.vertex_set[j]
        return euclidean(u, v)
    
    def reset(self):
        self.bead_set = np.array([(randint(low=1, high=100), randint(low=1, high=100)) for i in range(2)])
        self.defining_state = (randint(low=1, high=100), randint(low=1, high=100))
        self.answer = 0
        self.correct_answer = np.sqrt(self.defining_state[0]**2 + self.defining_state[1]**2)
    
    def step(self, action_index):
        # if distance should be measured
        #print("beads at start of step", self.bead_set)
        if (action_index == 8):
            self.answer = self.measure_distance(self.bead_set[0], self.bead_set[1])
        else:
            vertex_index = action_index % 4
            if (action_index < 5):
                # move first bead
                self.place_vertex(0, vertex_index)
            else:
                self.place_vertex(1, vertex_index)
        reward = 1 / (np.abs(self.answer - self.correct_answer)**2)
        done = self.answer == self.correct_answer
        #print("beads at end of step", self.bead_set)
        #print("distance: ", self.measure_distance(self.bead_set[0], self.bead_set[1]))
        return reward, done
    
    def get_state(self):
        a = np.array(self.defining_state)
        b = np.array(self.vertex_set[(self.bead_set[0])])
        c = np.array(self.vertex_set[self.bead_set[1]])
        nump = np.concatenate((a, b, c))
        t_state = torch.tensor(nump, dtype=torch.long)
        return t_state.flatten()
    
    def render(self):
        #print("Answer: ", self.answer)
        vertices = np.array([self.vertex_set[(self.bead_set[0])], self.vertex_set[(self.bead_set[1])]])
        #print(vertices)
        x, y = vertices.T
        plt.scatter(x, y)
        plt.show()

In [5]:
# REPLAY MEMORY DEFINITION
Transition = namedtuple('Transition', ('state', 'action', 'next_state', 'reward'))


class ReplayMemory(object):
    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)

Problems:

-Need to provide float args to vertex position (x and y)

-Need to provide index argument to vertex position

-Need to provide index arguments to distance measurement



In [6]:
# DEFINE THE NETWORK
# TODO: replace CNN with linear learner

class Actor(nn.Module):
    def __init__(self, input_dimension, output_dimension):
        super(Actor, self).__init__()

        self.fc1 = nn.Linear(input_dimension, 32)
        self.fc2 = nn.Linear(64, 32)
        self.head = nn.Linear(32, output_dimension)

    # Called with either one element to determine next action, or a batch
    # during optimization
    # Returns a distribution over the possible actions tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.float()
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.head(x.view(-1, 30)) # self.head(x.view(x.size(0), -1))

class Critic(nn.Module):
    def __init__(self, input_dimension, output_dimension):
        super(Critic, self).__init__()
        self.state1 = nn.Linear(input_dimension, 32)  # 6*6 from image dimension
        self.state2 = nn.Linear(32, 64)

        self.action1 = nn.Linear(3, 32)  # 6*6 from image dimension

        self.merged1 = nn.Linear(64, 32)

        self.head = nn.Linear(32, output_dimension)

    # Called with either one element to determine next action, or a batch
    # during optimization
    # Returns a distribution over the possible actions tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = x.float()
        #print(x.shape)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.head(x.view(-1, 30)) # self.head(x.view(x.size(0), -1))

"""
batch_size = 5
nb_classes = 2
in_features = 10

model = nn.Linear(in_features, nb_classes)
criterion = nn.CrossEntropyLoss()
"""

'\nbatch_size = 5\nnb_classes = 2\nin_features = 10\n\nmodel = nn.Linear(in_features, nb_classes)\ncriterion = nn.CrossEntropyLoss()\n'

In [23]:
import random
from collections import deque

class ActorCritic:
    def __init__(self, env):
        self.env  = env

        self.action_dimension = 3
        self.environment_dimension = 6     
        
        self.learning_rate = 0.001
        self.epsilon = 1.0
        self.epsilon_decay = .995
        self.gamma = .95
        self.tau   = .125
        self.memory = deque(maxlen=2000)

        # ===================================================================== #
        #                               Actor Model                             #
        # Chain rule: find the gradient of changing the actor network params in #
        # getting closest to the final value network predictions, i.e. de/dA    #
        # Calculate de/dA as = de/dC * dC/dA, where e is error, C critic, A act #
        # ===================================================================== #

        self.memory = deque(maxlen=2000)

        self.actor_model = Actor(self.action_dimension, self.environment_dimension).float()
        self.target_actor_model = Actor(self.action_dimension, self.environment_dimension).float()

        self.target_actor_model.load_state_dict(self.actor_model.state_dict())
        self.target_actor_model.eval()

        self.actor_optimizer = optim.Adam(self.actor_model.parameters())
        
        """
        self.actor_critic_grad = tf.placeholder(tf.float32, 
            [None, self.env.action_space.shape[0]]) # where we will feed de/dC (from critic)
        
        actor_model_weights = self.actor_model.trainable_weights
        self.actor_grads = tf.gradients(self.actor_model.output, 
            actor_model_weights, -self.actor_critic_grad) # dC/dA (from actor)
        
        grads = zip(self.actor_grads, actor_model_weights)

        
        self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)
         optimizer = optim.Adam(policy_net.parameters())
         """

        # ===================================================================== #
        #                              Critic Model                             #
        # ===================================================================== #        

        self.critic_model = Critic(self.action_dimension + self.environment_dimension, 1).float()
        self.target_critic_model = Critic(self.action_dimension + self.environment_dimension, 1).float()

        self.target_critic_model.load_state_dict(self.critic_model.state_dict())
        self.target_critic_model.eval()

        self.critic_optimizer = optim.Adam(self.actor_model.parameters())

        """
        self.critic_grads = tf.gradients(self.critic_model.output, self.critic_action_input) # where we calcaulte de/        dC for feeding above
        optimizer = optim.Adam(policy_net.parameters())
        """
        
    # ===================================================================== #
    #                              Training                                 #
    # ===================================================================== #    

    def remember(self, cur_state, action, reward, new_state, done):
        self.memory.append([cur_state, action, reward, new_state, done])
    
    def _train_actor(self, samples):
        for sample in samples:
            cur_state, action, reward, new_state, _ = sample
            predicted_action = self.actor_model(cur_state)
            
            grads = torch.autograd.grad(self.critic_model, torch.tensor([cur_state, predicted_action]))
            
            # optimize actor
            self.optimize_actor(cur_state, grads)
            """
            grads = self.sess.run(self.critic_grads, feed_dict={
                self.critic_state_input:  cur_state,
                self.critic_action_input: predicted_action
            })[0]

            self.sess.run(self.optimize, feed_dict={
                self.actor_state_input: cur_state,
                self.actor_critic_grad: grads
            })
            """
            
    def _train_critic(self, samples):
        for sample in samples:
            cur_state, action, reward, new_state, done = sample
            if not done:
                target_action = self.target_actor_model(new_state)
                # include predicted future reward in reward
                future_reward = self.target_critic_model([new_state, target_action])[0]
                reward += self.gamma * future_reward
            # train model
            self.optimize_critic([cur_state, action], reward)
    
    def optimize_critic(self, input_state, reward):
        # optimize critic on a batch

        # get critic predictions
        expected_rewards = self.critic_model(input_state)

        # Compute loss
        loss = F.smooth_l1_loss(expected_rewards, reward.unsqueeze(1))

        # Optimize the model
        self.critic_optimizer.zero_grad()
        loss = loss.double()
        loss.backward()

        for param in self.critic_model.parameters():
            param.grad.data.clamp_(-1, 1)

        optimizer.step()
    
    def optimize_actor(self, input, reward):
        expected_state_action_values = (next_state_values * GAMMA) + reward_batch

        loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

        # Optimize the model
        optimizer.zero_grad()
        loss = loss.double()
        loss.backward()

        for param in policy_net.parameters():
            param.grad.data.clamp_(-1, 1)

        optimizer.step()
        
    def train(self):
        batch_size = 32
        if len(self.memory) < batch_size:
            return

        rewards = []
        samples = random.sample(self.memory, batch_size)
        self._train_critic(samples)
        self._train_actor(samples)

    # ========================================================================= #
    #                         Target Model Updating                             #
    # ========================================================================= #

    def _update_actor_target(self):
        self.target_actor_model.load_state_dict(self.actor_model.state_dict())

    def _update_critic_target(self):
        self.target_critic_model.load_state_dict(self.critic_model.state_dict())

    def update_target(self):
        self._update_actor_target()
        self._update_critic_target()

    # ========================================================================= #
    #                              Model Predictions                            #
    # ========================================================================= #

    def act(self, cur_state):
        self.epsilon *= self.epsilon_decay

        if np.random.random() < self.epsilon:
            return self.env.action_space.sample()
        
        return self.actor_model.predict(cur_state)

In [24]:
def main():
    env = GeometryEnvironment()
    actor_critic = ActorCritic(env)

    num_trials = 10000
    trial_len  = 500

    cur_state = env.reset()
    action = env.action_space.sample()
    while True:
        #env.render()
        cur_state = env.get_state()

        action = actor_critic.act(cur_state)
        #action = action.reshape((1, env.action_space.shape[0]))

        new_state, reward, done, _ = env.step(action)
        #new_state = new_state.reshape((1, env.observation_space.shape[0]))

        actor_critic.remember(cur_state, action, reward, new_state, done)
        actor_critic.train()

        cur_state = new_state

In [25]:
main()

AttributeError: 'GeometryEnvironment' object has no attribute 'action_space'

In [26]:
# DEFINE TRANSITION OPERATIONS
BATCH_SIZE = 128
GAMMA = 0.999
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 200
TARGET_UPDATE = 10

# Get number of actions from gym action space
n_actions = 3

actor_net = Actor(n_actions).to(device)

critic_net = Critic(n_actions).to(device)


target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

# convert to double?
policy_net = DQN(n_actions).float()
target_net = DQN(n_actions).float()

optimizer = optim.Adam(policy_net.parameters())
memory = ReplayMemory(10000)


steps_done = 0


def select_action(state):
    global steps_done
    sample = random.random()
    eps_threshold = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * steps_done / EPS_DECAY)
    steps_done += 1
    if sample > eps_threshold:
        with torch.no_grad():
            # t.max(1) will return largest column value of each row.
            # second column on max result is index of where max element was
            # found, so we pick action with the larger expected reward.
            """
            best_action = policy_net(state)
            best_action = best_action.max(1)
            best_action = best_action[1]
            best_action.view(1, 1)
            """
            return  policy_net(state).max(1)[1].view(1, 1)
    else:
        return torch.tensor([[random.randrange(n_actions)]], device=device, dtype=torch.long)


episode_durations = []


def plot_durations():
    """
    plt.figure(2)
    plt.clf()
    durations_t = torch.tensor(episode_durations, dtype=torch.float)
    plt.title('Training...')
    plt.xlabel('Episode')
    plt.ylabel('Duration')
    plt.plot(durations_t.numpy())
    # Take 100 episode averages and plot them too
    if len(durations_t) >= 100:
        means = durations_t.unfold(0, 100, 1).mean(1).view(-1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy())

    plt.pause(0.001)  # pause a bit so that plots are updated
    if is_ipython:
        display.clear_output(wait=True)
        display.display(plt.gcf())
    """



TypeError: __init__() missing 1 required positional argument: 'output_dimension'

In [27]:
# TRAINING FUNCTION
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return
    transitions = memory.sample(BATCH_SIZE)
    # Transpose the batch (see https://stackoverflow.com/a/19343/3343043 for
    # detailed explanation). This converts batch-array of Transitions
    # to Transition of batch-arrays.
    batch = Transition(*zip(*transitions))
    #print(batch)

    # Compute a mask of non-final states and concatenate the batch elements
    # (a final state would've been the one after which simulation ended)
    non_final_mask = torch.tensor(tuple(map(lambda s: s is not None,
                                          batch.next_state)), device=device, dtype=torch.bool)
    non_final_next_states = torch.cat([s for s in batch.next_state
                                                if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model computes Q(s_t), then we select the
    # columns of actions taken. These are the actions which would've been taken
    # for each batch state according to policy_net
    state_batch = state_batch.view(BATCH_SIZE, 6)
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Compute V(s_{t+1}) for all next states.
    # Expected values of actions for non_final_next_states are computed based
    # on the "older" target_net; selecting their best reward with max(1)[0].
    # This is merged based on the mask, such that we'll have either the expected
    # state value or 0 in case the state was final.
    non_final_next_states = non_final_next_states.view(non_final_next_states.shape[0] // 6, 6)

    next_state_values = torch.zeros(BATCH_SIZE, device=device)
    next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0].detach()
    # Compute the expected Q values
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    # Compute Huber loss
    state_action_values = state_action_values.double()
    expected_state_action_values = expected_state_action_values.double()
    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()
    loss = loss.double()
    loss.backward()
    for param in policy_net.parameters():
        param.grad.data.clamp_(-1, 1)
    optimizer.step()

In [28]:
# TRAINING LOOP
num_episodes = 50
for i_episode in range(num_episodes):
    # Initialize the environment and state
    env.reset()
    state = env.get_state()
    for t in count():
        print(t)
        #print(state)
        # Select and perform an action
        action = select_action(state)
        reward, done = env.step(action.item())
        reward = torch.tensor([reward], device=device)

        # Observe new state
        if not done:
            next_state = env.get_state()
        else:
            next_state = None

        # Store the transition in memory
        memory.push(state, action, next_state, reward)

        # Move to the next state
        state = next_state

        # Perform one step of the optimization (on the target network)
        optimize_model()
        if done:
            episode_durations.append(t + 1)
            plot_durations()
            break
    # Update the target network, copying all weights and biases in DQN
    if i_episode % TARGET_UPDATE == 0:
        target_net.load_state_dict(policy_net.state_dict())

print('Complete')
env.render()
#env.close()
plt.ioff()
plt.show()

NameError: name 'env' is not defined

In [29]:
test_states = []
test_actions = []

env.reset()
state = env.get_state()
for t in count():
    print(t)
    #print(state)
    # Select and perform an action
    action = select_action(state)
    reward, done = env.step(action.item())
    reward = torch.tensor([reward], device=device)

    test_states.append(state)
    test_actions.append(action)

    # Observe new state
    if not done:
        next_state = env.get_state()
    else:
        next_state = None


    # Move to the next state
    state = next_state

    if done:
        episode_durations.append(t + 1)
        plot_durations()
        break

NameError: name 'env' is not defined

In [30]:
import matplotlib.animation as animation
from IPython.display import HTML

measurements = np.array(test_actions) == 8
#=========================================
# Animate Images

fig = plt.figure(figsize=(10, 10))
ims = []
for i in range(len(test_states)):
    x = np.array(test_states[i])[2:5:2]
    y = np.array(test_states[i])[3:6:2]
    if (measurements[i]):
        ims.append([plt.scatter(x, y, animated=True)])
        im, = plt.plot(x, y, 'ro-', animated=True)
    else:
        im = plt.scatter(x, y, animated=True)
    ims.append([im])

ani = animation.ArtistAnimation(fig, ims, interval=300, blit=True, repeat_delay=1000)
plt.close()

# Show the animation
HTML(ani.to_jshtml())